In [1]:
from Utils import pdb_to_uniprot
from Preprocessing.Download import download_prot_structure
from Preprocessing.ESM import get_esm, get_protein_esm
from Preprocessing.GraphProcessing import PairMoleculeDataset
from Training import Trainer
from config import Config
import pandas as pd
from tqdm import tqdm
import time
import glob
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
config = Config()

Transferred model to cuda:0


DEBUG:matplotlib:matplotlib data path: /home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/mkpandey/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', '_distutils_hack', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'itertools', 'reprlib', '_collections', 'collections', '_functools', 'functo

To use the Graphein submodule graphein.protein.features.sequence.embeddings, you need to install biovec.

biovec cannot be installed via conda


DEBUG:matplotlib:CACHEDIR=/home/mkpandey/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/mkpandey/.cache/matplotlib/fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


To use the Graphein submodule graphein.protein.visualisation, you need to install pytorch3d.

To do so, use the following command:

    conda install -c pytorch3d pytorch3d
To use the Graphein submodule graphein.protein.meshes, you need to install pytorch3d.

To do so, use the following command:

    conda install -c pytorch3d pytorch3d
To use the Graphein submodule graphein.ml.conversion, you need to install dgl.

To do so, use the following command:

    conda install -c dglteam dgl
To use the Graphein submodule graphein.protein.features.nodes.aaindex, you need to install pyaaisc.

pyaaisc cannot be installed via conda
Torch version: 1.9.0
Cuda available: True
Torch geometric version: 2.0.3


# Make ML pyG dataset 

Documentation asking user to put DPI dataset into data folder with /raw/filename.csv structure.

In [2]:
#1. Read your data file
dpi_data_path,filename = './data/DPI/PDBBind_Sample/','sample_pdbbind_dataset'
df_input = pd.read_csv(f'{dpi_data_path}/raw/{filename}.csv')
df_input.to_csv(config.root+config.project_name+'/raw/'+filename+'.csv')
print(df_input.shape)
df_input.head()

(31, 4)


,pdbid,label,SMILES,Interaction_ID
0,4eoh,4.30,Cn1c(=O)c2[nH]cnc2n(C)c1=O,0
1,3zc5,7.13,C[C@@H](c1ccc2ncccc2c1)c1nnc2ccc(-c3cnn(C)c3)nn12,1
2,1utl,2.47,[NH3+]CCCc1ccccc1,2
3,1p1n,6.80,C=C(C)[C@H]1C[NH2+][C@H](C(=O)[O-])[C@H]1CC(=O...,3
4,2y7i,5.54,NC(=[NH2+])NCCC[C@H]([NH3+])C(=O)[O-],4


In [3]:
# 2. Process structures and esm embedding
# #download pdb structures for pdbid in the dataset. When pdb structures are not present, download alphafold

config.download_structures = True

if config.download_structures:
    pdb_ids = df_input.pdbid.unique()
    pdbid_to_structure, pdbid_to_esm =dict(),dict()
    for pdb_id in tqdm(pdb_ids):
        try:
            pdbid_to_structure[pdb_id] = download_prot_structure(pdb_id,struct_type='pdb',verbose=False)
        except Exception as e:
            print(i,e)
            pdbid_to_structure[pdb_id]= download_prot_structure(code = pdb_to_uniprot(pdb_id),struct_type='af')
        
    df_input['path']=''
    
    for pdb_id in tqdm(pdb_ids,desc='ESM Calculation'):
        pdbid_to_esm[pdb_id] = get_protein_esm(pdbid_to_structure[pdb_id])
        
    
    for i in tqdm(range(len(df_input))):
        df_input.iat[i,4] = pdbid_to_structure[df_input.pdbid[i]]
        

        
    df_input.to_csv(f'{config.root+config.project_name}/raw/{filename}_w_struct_path.csv')
    with open(config.esm_file,'wb') as f:
        pickle.dump(pdbid_to_esm,f)
    

df_input.head()

ESM Calculation:   0%|          | 0/30 [00:00<?, ?it/s]/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5500.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5569.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5675.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5833.
  warnings.warn(
ESM Calculation:  10%|█         | 3/30 [00:01<00:09,  2.90it/s]/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Cha

ESM Calculation:  80%|████████  | 24/30 [00:03<00:00,  8.92it/s]/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 13653.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 13705.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 13737.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 14295.
  warnings.warn(
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5858.
  warnings.war

,pdbid,label,SMILES,Interaction_ID,path
0,4eoh,4.30,Cn1c(=O)c2[nH]cnc2n(C)c1=O,0,.//psgbar_expt_pdbbind/data/prot/PDB/4eoh.pdb
1,3zc5,7.13,C[C@@H](c1ccc2ncccc2c1)c1nnc2ccc(-c3cnn(C)c3)nn12,1,.//psgbar_expt_pdbbind/data/prot/PDB/3zc5.pdb
2,1utl,2.47,[NH3+]CCCc1ccccc1,2,.//psgbar_expt_pdbbind/data/prot/PDB/1utl.pdb
3,1p1n,6.80,C=C(C)[C@H]1C[NH2+][C@H](C(=O)[O-])[C@H]1CC(=O...,3,.//psgbar_expt_pdbbind/data/prot/PDB/1p1n.pdb
4,2y7i,5.54,NC(=[NH2+])NCCC[C@H]([NH3+])C(=O)[O-],4,.//psgbar_expt_pdbbind/data/prot/PDB/2y7i.pdb


In [4]:
#3. Create pairmolecule dataset for your data on first run
config.create_dataset = True
if config.create_dataset:
    dataset = PairMoleculeDataset(root=config.root+config.project_name,filename = 'sample_pdbbind_dataset_w_struct_path.csv',
                              subsample=None,labelcol='label')


esm loaded
csv read and filtered 31


In [5]:
processed_interactions = [int(f.split('_')[-1].split('.')[0]) for f in glob.glob(f'{config.root+config.project_name}/processed/data_*.pt')]
df = df_input[df_input['Interaction_ID'].isin(processed_interactions)]
print(df.shape)
df.head(2)

(31, 5)


,pdbid,label,SMILES,Interaction_ID,path
0,4eoh,4.30,Cn1c(=O)c2[nH]cnc2n(C)c1=O,0,.//psgbar_expt_pdbbind/data/prot/PDB/4eoh.pdb
1,3zc5,7.13,C[C@@H](c1ccc2ncccc2c1)c1nnc2ccc(-c3cnn(C)c3)nn12,1,.//psgbar_expt_pdbbind/data/prot/PDB/3zc5.pdb


#  Make Train and test dataset with sklearn split, right kind of fold, cross validation etc. 

In [6]:
df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
print(df_train.shape, df_test.shape)
temp_train_df = pd.DataFrame(zip(df_train.Interaction_ID), columns=['Interaction_ID'])
temp_test_df = pd.DataFrame(zip(df_test.Interaction_ID), columns=['Interaction_ID'])
temp_train_df.to_csv(config.root+config.project_name+'/raw/temp_train.csv')
temp_test_df.to_csv(config.root+config.project_name+'/raw/temp_test.csv')
t0 = time.time()
train_dataset = PairMoleculeDataset(root = config.root+config.project_name, labelcol='label', filename = 'temp_train.csv')
print(time.time()-t0) #1016.2
test_dataset = PairMoleculeDataset(root = config.root+config.project_name, labelcol='pKd', filename = 'temp_test.csv')
train_dataset,test_dataset

(24, 5) (7, 5)
esm loaded
csv read and filtered 24
0.006377696990966797
esm loaded
csv read and filtered 7


(PairMoleculeDataset(24), PairMoleculeDataset(7))

In [7]:
Trainer.train_model(train_dataset,test_dataset)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
wandb: Currently logged in as: diamondspark (use `wandb login --relogin` to force relogin)


train dataloader time 0.0002651214599609375


DEBUG:git.cmd:Popen(['git', 'cat-file', '--batch-check'], cwd=/home/mkpandey/MyProjects/Drug_Protein_Interaction_Project/ER_AR_project/PSG_BAR_Experiments/graph-binding-affinity-new-interact-attention-gitable, universal_newlines=False, shell=None, istream=<valid stream>)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


super-resonance-3


100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


...

VAL: Epoch 5| Average Val Loss 11.580165502843732 time 2022-04-22 10:08:00.643553
{'average loss test': 11.580165502843732, 'pearson correlation': 0.16605956314734868}


  0%|          | 0/1 [00:00<?, ?it/s]


/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mkpandey/anaconda3/envs/psg-bar/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


average loss test,█▂▂▁▁▂
average loss train,█▅▁▁▁▁
epoch,▁▂▄▅▇█
pearson correlation,▁▁▃▅▇█
architecture,MMProtGraph( (smil...
average loss test,11.58017
average loss train,3.83406
epoch,5
pearson correlation,0.16606


KeyboardInterrupt: 